In [ ]:
import numpy as np
import pandas as pd

#描画用ライブラリ
import matplotlib.pyplot as plt
import holoviews as hv

import glob
import os

#信号処理用
from scipy import fftpack
from scipy import signal
import scipy.signal as signal
import wave
import random

In [ ]:
#wave dataの読み込み
def wave_load(filename):
    # open wave file
    wf = wave.open(filename,'r')
    channels = wf.getnchannels()

    # load wave data
    chunk_size = wf.getnframes()
    amp  = (2**8) ** wf.getsampwidth() / 2
    data = wf.readframes(chunk_size)   # バイナリ読み込み
    data = np.frombuffer(data,'int16') # intに変換
    #data = data / amp                  # 振幅正規化
    data = data[::channels]
    samplingRate = wf.getframerate()
    frameNum = wf.getnframes()
    return data, samplingRate,frameNum

In [ ]:
#wavデータの保存フォルダの指定
DIR_TRAIN_NORMAL =r"..\train_normal\train_normal"
DIR_VALID_ANORMAL =r"..\valid_anomaly\valid_anomaly"
DIR_VALID_TEST   = r"..\test\test"

In [ ]:
NomalDataNum = len(glob.glob(os.path.join(DIR_TRAIN_NORMAL, "*.wav")))
AnomalyDataNum = len(glob.glob(os.path.join(DIR_TRAIN_NORMAL, "*.wav")))
testDataNum = len(glob.glob(os.path.join(DIR_TRAIN_NORMAL, "*.wav")))

In [ ]:
#サンプリングレート/フレームサイズを取得．全データ同一であることが前提
data, samplingRate,frameNum = wave_load(glob.glob(os.path.join(DIR_TRAIN_NORMAL, "*.wav"))[1] )

In [ ]:
#読み込み用葉配列初期化
data_train_normal_raw = np.zeros((NomalDataNum,frameNum ))
data_train_anomaly_raw =  np.zeros((AnomalyDataNum,frameNum ))
data_test_raw =  np.zeros((testDataNum,frameNum ))
#前処理　エンベロープ処理データ用
data_train_normal_enb = np.zeros((NomalDataNum,frameNum ))
data_train_anomaly_enb =  np.zeros((AnomalyDataNum,frameNum ))
data_test_enb =  np.zeros((testDataNum,frameNum ))

In [ ]:
# 時間軸の作成
dt = 1/samplingRate
timeRange = dt*(frameNum)
t = np.arange(0, timeRange, dt)

i=0
for f in glob.glob(os.path.join(DIR_TRAIN_NORMAL, "*.wav")):
    fileName = f
    data, samplingRate,frameNum  = wave_load(fileName)
    data_train_normal_raw[i,:] = data
    data_train_normal_enb[i,:] = np.abs(signal.hilbert(data))
    i=i+1
        
i=0
for f in glob.glob(os.path.join(DIR_VALID_ANORMAL, "*.wav")):
    fileName = f
    data, samplingRate,frameNum  = wave_load(fileName)
    data_train_anomaly_raw[i,:] = data
    data_train_anomaly_enb[i,:] = np.abs(signal.hilbert(data))
    i=i+1
    
i=0
for f in glob.glob(os.path.join(DIR_VALID_TEST, "*.wav")):
    fileName = f
    data, samplingRate,frameNum  = wave_load(fileName)
    data_test_raw[i,:] = data
    data_test_enb[i,:] = np.abs(signal.hilbert(data))
    i=i+1

In [ ]:
#読み込むデータの開始位置とデータサイズと間引き率の設定
dataStart = 0
dataLength = 84000
downSaplingRate = 7

# Input Option 1. Flat DataFrame or Wide DataFrame
データフレームの形状
id	time	x	       y
A	t1	x(A, t1)	y(A, t1)
A	t2	x(A, t2)	y(A, t2)
A	t3	x(A, t3)	y(A, t3)
B	t1	x(B, t1)	y(B, t1)
B	t2	x(B, t2)	y(B, t2)
B	t3	x(B, t3)	y(B, t3)

In [ ]:
#tsfresh入力用に配列を並び替え
df_nomal = pd.DataFrame(data_train_normal_enb[:,:])          #pandas dataFrame化
df_nomal_data = pd.DataFrame([[0,0]],columns=["value","id"]) #格納用dataFrameの初期化
for i in range(df_nomal.shape[0]):
    df_temp = pd.DataFrame( df_nomal.iloc[i,dataStart:dataStart+dataLength:downSaplingRate] )
    df_temp = df_temp.rename(columns={i: 'value'})
    df_temp = df_temp.assign(id=i)
    df_nomal_data = pd.concat([df_nomal_data, df_temp], axis=0)

df_nomal_data = df_nomal_data.reset_index()
df_nomal_data = df_nomal_data.drop(0, axis=0)

In [ ]:
df_anomaly = pd.DataFrame(data_train_anomaly_enb[:,:])
df_anomaly_data = pd.DataFrame([[0,0]],columns=["value","id"])
for i in range(df_anomaly.shape[0]):
    df_temp = pd.DataFrame( df_anomaly.iloc[i,dataStart:dataStart+dataLength:downSaplingRate] )
    df_temp = df_temp.rename(columns={i: 'value'})
    df_temp = df_temp.assign(id=i)
    
    df_anomaly_data = pd.concat([df_anomaly_data, df_temp], axis=0)
    
df_anomaly_data = df_anomaly_data.reset_index()
df_anomaly_data = df_anomaly_data.drop(0, axis=0)

In [ ]:
df_test = pd.DataFrame(data_test_enb[:,:])
df_test_data = pd.DataFrame([[0,0]],columns=["value","id"])
for i in range(df_test.shape[0]):
    df_temp = pd.DataFrame( df_test.iloc[i,dataStart:dataStart+dataLength:downSaplingRate] )
    df_temp = df_temp.rename(columns={i: 'value'})
    df_temp = df_temp.assign(id=i)
    
    df_test_data = pd.concat([df_test_data, df_temp], axis=0)
    
df_test_data = df_test_data.reset_index()
df_test_data = df_test_data.drop(0, axis=0)

In [ ]:
df_test_data

In [ ]:
#data保存名の指定
resultFileName_nomal = "nomal.pkl"
resultFileName_anomaly = "anomaly.pkl"
resultFileName_test = "test.pkl"

In [ ]:
import tsfresh.feature_extraction.feature_calculators as feature_calculators
from tsfresh import extract_features

#extracted_features = extract_features(temp, column_id='id')
extracted_features = extract_features(df_anomaly_data, column_id='id',column_sort='index')


In [ ]:
display(extracted_features.columns)

In [ ]:
#計算結果の保存
pd.to_pickle(extracted_features,resultFileName_anomaly)

In [ ]:
import tsfresh.feature_extraction.feature_calculators as feature_calculators
from tsfresh import extract_features

#extracted_features = extract_features(temp, column_id='id')
extracted_features = extract_features(df_nomal_data, column_id='id',column_sort='index')


In [ ]:
pd.to_pickle(extracted_features,resultFileName_nomal)

In [ ]:
import tsfresh.feature_extraction.feature_calculators as feature_calculators
from tsfresh import extract_features

#extracted_features = extract_features(temp, column_id='id')
extracted_features = extract_features(df_test_data, column_id='id',column_sort='index')

In [ ]:
pd.to_pickle(extracted_features,resultFileName_nomal)

In [ ]:
#可視化用　データの読み込み

In [ ]:
if 0:
    hv.extension('bokeh')
    current_graph=hv.Overlay([hv.Curve((time,data_train_normal_raw[44,:])),hv.Curve((time,data_train_normal_enb[44,:]))])

    # 表示
    current_graph

    # タイトルの設定
    current_graph.opts(title="enbelope")
    # グリッドの描画
    current_graph.opts(show_grid=True)
    # 軸ラベルの変更
    current_graph.opts(xlabel="time", ylabel="signel")

    # 描画サイズの変更
    current_graph.opts(width=1280, height=640)

    # グラフをhtmlで保存
    renderer = hv.renderer('bokeh')
    renderer.save(current_graph, "enbelope")
    #renderer.save(current_graph, "enbelope.png")